In [1]:
import os, base64, asyncio, json, requests
from openai import AzureOpenAI
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
# from autogen_ext.teams.magentic_one import MagenticOneGroupChat
from autogen_core.models import UserMessage
from autogen_agentchat.messages import UserMessage
from autogen_agentchat.agents import AssistantAgent
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField, 
    SearchFieldDataType,
    SimpleField,
    HnswAlgorithmConfiguration,
    VectorSearchProfile
)
# from utility.llm_config import llm_config
    
from dotenv import load_dotenv

load_dotenv()

# Retrieve credentials from environment variables
azure_openAI_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
azure_openAI_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openAI_deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openAI_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")

azure_search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
azure_search_key = os.environ.get("AZURE_SEARCH_KEY")
azure_search_deployment_name = os.environ.get("AZURE_SEARCH_DEPLOYMENT_NAME") 

vector_dimension = 1536  # Dimension for text-embedding-3-small
datafile = "./Data"
azure_search_index_name = "rag-1756588180789"


In [2]:
from _tools._searchDocuments import search_documents

search_documents("who is Mohammad?")

'\nResult 1:\nPTL - WZ - Home Support Requests\n*** Note: No case manager / csp coordinator will be noted in Western – unless the patient already had an agency.\n🤹️ Actions to Follow:\n· Hit "Send" under Plan Summary\n· Go to Prod Folder to obtain documentation\n· Obtain the partially completed EAN form\n· Update section related to hours in the format of: " #hours /day, #days /week"\n· Update section related to "Agency/Bookkeeper" to say " To be coordinated by DischargeHUB"\n· Email the updated EAN form to the referring Social Worker\n· Once the Social Worker has updated, they will copy dischargeHUB@nlhealthservices.ca on their email to csrequests@nlhealthservices.ca\n· PTL to send out Template A/B/C - based on requirement of referral\n· Update portal to reflect when initial outreach was made\n· " - date + time + initials\n· If no response from agencies within a 2-hour window - begin calling - random order \n· When an agency has been found - two possible scenarios for next steps\n· Age

In [ ]:
from _clients._azureOpenAIChatCompletion_client import model_client
import requests
from _clients._openAI_client import openAI_client



def embed_text(text):
    response = openAI_client.embeddings.create(
        input=[text],
        model=azure_search_deployment_name
    )
    embedding = response.data[0].embedding
    return embedding



def search_documents(query=[], top_k=3):

    embedding = embed_text(query)

    url= f"{azure_search_endpoint}/indexes/{azure_search_index_name}/docs/search?api-version={'2023-07-01-Preview'}"

    headers = {
        "Content-Type": "application/json",
        "api-key": azure_search_key
    }

    payload = {
        "count": True,
        "search":"{query}",
        "vector": {
            "value": embedding,
            "k": top_k,
            "fields": "text_vector"
        },
        "top": top_k
    }

    response = requests.post(url, headers=headers, json=payload)
    response = response.json().get("value", [])
    response = [doc.get("chunk") for doc in response if doc.get("chunk")]

    return response


userQuery = "waht is template J for?"

result = search_documents(userQuery)





0
PTL - WZ - Home Support Requests
*** Note: No case manager / csp coordinator will be noted in Western – unless the patient already had an agency.
🤹️ Actions to Follow:
· Hit "Send" under Plan Summary
· Go to Prod Folder to obtain documentation
· Obtain the partially completed EAN form
· Update section related to hours in the format of: " #hours /day, #days /week"
· Update section related to "Agency/Bookkeeper" to say " To be coordinated by DischargeHUB"
· Email the updated EAN form to the referring Social Worker
· Once the Social Worker has updated, they will copy dischargeHUB@nlhealthservices.ca on their email to csrequests@nlhealthservices.ca
· PTL to send out Template A/B/C - based on requirement of referral
· Update portal to reflect when initial outreach was made
· " - date + time + initials
· If no response from agencies within a 2-hour window - begin calling - random order 
· When an agency has been found - two possible scenarios for next steps
· Agency found before* *the appr

c:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


In [ ]:
from _clients._azureOpenAIChatCompletion_client import model_client
import os, requests
import requests
from _clients._openAI_client import openAI_client
from pydantic import BaseModel, Field



# Define Pydantic model for a single integer output
class Output_ragResultEvaluator(BaseModel):
    value: int = Field(..., ge=0, le=100)  # Integer from 0 to 100



def embed_text(text):
    response = openAI_client.embeddings.create(
        input=[text],
        model=azure_search_deployment_name
    )
    embedding = response.data[0].embedding
    return embedding



def search_documents(query=[], top_k=3):

    embedding = embed_text(query)

    url= f"{azure_search_endpoint}/indexes/{azure_search_index_name}/docs/search?api-version={'2023-07-01-Preview'}"

    headers = {
        "Content-Type": "application/json",
        "api-key": azure_search_key
    }

    payload = {
        "count": True,
        "search":"{query}",
        "vector": {
            "value": embedding,
            "k": top_k,
            "fields": "text_vector"
        },
        "top": top_k
    }

    response = requests.post(url, headers=headers, json=payload)
    response = response.json().get("value", [])
    response = [doc.get("chunk") for doc in response if doc.get("chunk")]

    return response

result = search_documents("who is Mohammad?")

agent = AssistantAgent(
    name="RetrievedResultEvaluator",
    model_client= model_client,
    system_message="You are a helpful assistant. that get a user 'query' and a 'result'. I want you to compare the user query with the result and if the result helps to answer the user query, return an core between o to 100. 0 shows the least relevancy between query and result while 100 shows the mosts'",
    output_content_type=Output_ragResultEvaluator
)

result = result[2]

result_ev = await agent.run(task=f"query: who is Mohammad? result: {result}")
print(result_ev.messages[-1].content)  # Outputs the final summarized response



In [4]:
# ##### Assistant Agent
# assistant_bestFormatFinder  = AssistantAgent(
#     name='formatFinder', 
#     model_client=model_client,
#     system_message="you always say i donot know",
#     )

# assistant_findAnswer  = AssistantAgent(
#     name='answerFinder', 
#     model_client=model_client,
#     system_message="You find the answer to the user's question based on the provided context, tools and knowledge you have",
#     )

# assistant_evaliation = AssistantAgent(
#     name='evaluator', 
#     model_client=model_client,
#     system_message="You evaluate the final output based on the user's query, provided context and tools and your knowledge",
#     )   




In [5]:
# result= await assistant_evaliation.run(task="where is paris? answer in one word")
# print(result.messages[-1].content) 

### Udemy Course


In [6]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from dotenv import load_dotenv
load_dotenv()
from _clients._azureOpenAIChatCompletion_client import AzureOpenAIChatCompletionClient
 